# Part 2

## Load data

In [1]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np

def load_feature_groups(path):
    wb = load_workbook(path, data_only=True)
    ws = wb.active

    TARGET_COLOR = "FF5EB91E"  # green color used for good features

    def get_color(cell):
        if cell.fill.patternType is None:
            return None
        return cell.fill.fgColor.rgb

    good = []
    bad = []

    for row in ws.iter_rows(min_row=2, values_only=False):  # skip header
        feature_name = row[1].value      # column B = feature name in Dutch
        color = get_color(row[1])        

        if color == TARGET_COLOR:
            good.append(feature_name)
        else:
            bad.append(feature_name)

    return good, bad

good_features, bad_features = load_feature_groups("data/data_description_colored.xlsx")
good_features, bad_features


C:\Users\yutin\AppData\Local\Temp\ipykernel_2264\2302363106.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


(['adres_aantal_brp_adres',
  'adres_aantal_verschillende_wijken',
  'adres_aantal_verzendadres',
  'adres_aantal_woonadres_handmatig',
  'adres_dagen_op_adres',
  'afspraak_aanmelding_afgesloten',
  'afspraak_afgelopen_jaar_afsprakenplan',
  'afspraak_afgelopen_jaar_ontheffing',
  'afspraak_afgelopen_jaar_plan_van_aanpak',
  'afspraak_afgelopen_jaar_signaal_voor_medewerker',
  'afspraak_afgelopen_jaar_vervolgmeting_matchbaarheid_werkzoekende_klant',
  'afspraak_afgelopen_jaar_voortgang_aanmelding_en_deelname',
  'afspraak_afsprakenplan',
  'afspraak_controle_aankondiging_maatregel',
  'afspraak_controle_verwijzing',
  'afspraak_deelname_compleet_uit_webapplicatie',
  'afspraak_inspanningsperiode',
  'afspraak_laatstejaar_resultaat_ingevuld',
  'afspraak_laatstejaar_resultaat_ingevuld_uniek',
  'afspraak_other',
  'afspraak_participatietrede_vervolgmeting',
  'afspraak_resultaat_ingevuld_uniek',
  'afspraak_signaal_van_aanbieder',
  'afspraak_signaal_voor_medewerker',
  'afspraak_toevo

In [2]:
df_data = pd.read_csv("data/investigation_train_large_checked.csv")
df_data.head()

,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,...,typering_hist_ind,typering_hist_sector_zorg,typering_ind,typering_indicatie_geheime_gegevens,typering_other,typering_transport__logistiek___tuinbouw,typering_zorg__schoonmaak___welzijn,Ja,Nee,checked
0,1,1,0,0,23240,1,0,0,0,0,...,1,0,0,0,0,0,0,0.617698,0.382302,False
1,4,2,1,1,1971,1,0,0,1,0,...,1,0,1,0,1,0,0,0.602167,0.397833,False
2,6,4,2,1,7247,0,0,0,1,0,...,1,0,1,0,0,0,0,0.512377,0.487623,False
3,3,2,0,1,8060,1,0,0,1,0,...,1,0,0,0,0,0,0,0.717796,0.282204,True
4,3,2,0,0,18705,1,0,0,0,0,...,1,0,1,0,0,0,0,0.705484,0.294516,True


In [3]:
# Split bad features into groups: address-related, appointment-related, medical-related, competence-related, 
# language-related, gender-related, age-related, personal qualities-related, relationship-related
def partition_bad_features(bad_features):
    address_mask = ["adres"]
    appointment_mask = ["afspraak"]
    medical_mask = ["lichamelijke", "psychische", "verslaving", "medische", ]
    competence_mask = ["competentie"]
    language_mask = ["taal", "_nl_"]
    gender_mask = ["geslacht"]
    age_mask = ["leeftijd"]
    relationship_mask = ["sociaal", "sociale", "relatie"]
    # personal qualities: others

    groups = {
        "address": [],
        "appointment": [],
        "medical": [],
        "competence": [],
        "language": [],
        "gender": [],
        "age": [],
        "relationship": [],
        "personal_qualities": [],
    }

    # Helper function to check if any keyword is in the feature name
    def contains_keyword(feature, keywords):
        return any(keyword in feature.lower() for keyword in keywords)
    
    # Iterate through bad features and assign them to groups
    for feature in bad_features:
        if contains_keyword(feature, address_mask):
            groups["address"].append(feature)
        elif contains_keyword(feature, appointment_mask):
            groups["appointment"].append(feature)
        elif contains_keyword(feature, medical_mask):
            groups["medical"].append(feature)
        elif contains_keyword(feature, competence_mask):
            groups["competence"].append(feature)
        elif contains_keyword(feature, language_mask):
            groups["language"].append(feature)
        elif contains_keyword(feature, gender_mask):
            groups["gender"].append(feature)
        elif contains_keyword(feature, age_mask):
            groups["age"].append(feature)
        elif contains_keyword(feature, relationship_mask):
            groups["relationship"].append(feature)
        else:
            groups["personal_qualities"].append(feature)

    return groups

In [4]:
grouped_bad_features = partition_bad_features(bad_features)
grouped_bad_features

{'address': ['adres_recentst_onderdeel_rdam',
  'adres_recentste_buurt_groot_ijsselmonde',
  'adres_recentste_buurt_nieuwe_westen',
  'adres_recentste_buurt_other',
  'adres_recentste_buurt_oude_noorden',
  'adres_recentste_buurt_vreewijk',
  'adres_recentste_plaats_other',
  'adres_recentste_plaats_rotterdam',
  'adres_recentste_wijk_charlois',
  'adres_recentste_wijk_delfshaven',
  'adres_recentste_wijk_feijenoord',
  'adres_recentste_wijk_ijsselmonde',
  'adres_recentste_wijk_kralingen_c',
  'adres_recentste_wijk_noord',
  'adres_recentste_wijk_other',
  'adres_recentste_wijk_prins_alexa',
  'adres_recentste_wijk_stadscentru',
  'adres_unieke_wijk_ratio'],
 'appointment': ['afspraak_aantal_woorden',
  'afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel',
  'afspraak_afgelopen_jaar_ontheffing_taaleis',
  'afspraak_galo_gesprek',
  'afspraak_gespr__einde_zoekt___galo_gesprek_',
  'afspraak_laatstejaar_aantal_woorden'],
 'medical': ['belemmer

In [5]:
# Check how many unique values each bad feature has, in each group
grouped_feature_unique_values = {}
for group, features in grouped_bad_features.items():
    feature_unique_values = {}
    for feature in features:
        unique_values = df_data[feature].nunique()
        feature_unique_values[feature] = unique_values
    grouped_feature_unique_values[group] = feature_unique_values

grouped_feature_unique_values

{'address': {'adres_recentst_onderdeel_rdam': 2,
  'adres_recentste_buurt_groot_ijsselmonde': 2,
  'adres_recentste_buurt_nieuwe_westen': 2,
  'adres_recentste_buurt_other': 2,
  'adres_recentste_buurt_oude_noorden': 2,
  'adres_recentste_buurt_vreewijk': 2,
  'adres_recentste_plaats_other': 2,
  'adres_recentste_plaats_rotterdam': 2,
  'adres_recentste_wijk_charlois': 2,
  'adres_recentste_wijk_delfshaven': 2,
  'adres_recentste_wijk_feijenoord': 2,
  'adres_recentste_wijk_ijsselmonde': 2,
  'adres_recentste_wijk_kralingen_c': 2,
  'adres_recentste_wijk_noord': 2,
  'adres_recentste_wijk_other': 2,
  'adres_recentste_wijk_prins_alexa': 2,
  'adres_recentste_wijk_stadscentru': 2,
  'adres_unieke_wijk_ratio': 2},
 'appointment': {'afspraak_aantal_woorden': 967,
  'afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel': 2,
  'afspraak_afgelopen_jaar_ontheffing_taaleis': 2,
  'afspraak_galo_gesprek': 4,
  'afspraak_gespr__einde_zoekt___galo_gesprek

### For each feature, divide into groups

In [6]:
# For each group, calculate the lowest number of unique values, it will be used as the number of partitions in each group
# Except for age, since age is continuous, we can set a fixed number of partitions, e.g. 2
partition_counts = {}
for group, feature_unique_values in grouped_feature_unique_values.items():
    if group == "age":
        partition_counts[group] = 2
    else:
        min_unique_values = min(feature_unique_values.values()) if feature_unique_values else 0
        partition_counts[group] = min_unique_values
partition_counts


{'address': 2,
 'appointment': 2,
 'medical': 2,
 'competence': 2,
 'language': 2,
 'gender': 2,
 'age': 2,
 'relationship': 2,
 'personal_qualities': 2}

In [7]:
# Create partitions on df data for every bad feature based on its group partition count
def create_partitions_for_bad_features(df, grouped_bad_features, partition_counts):
    partitions = {}
    for group, features in grouped_bad_features.items():
        partitions[group] = {}
        num_partitions = partition_counts[group]

        for feature in features:
            if feature not in df.columns:
                partitions[group][feature] = []
                continue

            unique_values = df[feature].nunique()

            # if the number of unique values is less than or equal to the number of partitions, use unique values as partitions
            if unique_values <= num_partitions:
                partition_values = sorted(df[feature].dropna().unique().tolist())

            else:
                # Try qcut to create partitions using quantiles
                bins = pd.qcut(df[feature], q=num_partitions, duplicates='drop')
                partition_values = sorted(bins.unique().tolist(), key=lambda x: str(x))

                # If qcut failed to produce enough partitions
                if len(partition_values) < num_partitions:
                    # Fallback to using value-based partitions
                    unique_values = sorted(df[feature].dropna().unique().tolist())

                    # If still not enough, use equal-width binning
                    if len(unique_values) < num_partitions:
                        # Force equal-width binning
                        partition_values = unique_values
                    else:
                        # Create equal-width bins
                        bins = pd.cut(df[feature], bins=num_partitions)
                        partition_values = bins.unique().tolist()

            partitions[group][feature] = partition_values
    return partitions

feature_partitions = create_partitions_for_bad_features(df_data, grouped_bad_features, partition_counts)
feature_partitions

{'address': {'adres_recentst_onderdeel_rdam': [0, 1],
  'adres_recentste_buurt_groot_ijsselmonde': [0, 1],
  'adres_recentste_buurt_nieuwe_westen': [0, 1],
  'adres_recentste_buurt_other': [0, 1],
  'adres_recentste_buurt_oude_noorden': [0, 1],
  'adres_recentste_buurt_vreewijk': [0, 1],
  'adres_recentste_plaats_other': [0, 1],
  'adres_recentste_plaats_rotterdam': [0, 1],
  'adres_recentste_wijk_charlois': [0, 1],
  'adres_recentste_wijk_delfshaven': [0, 1],
  'adres_recentste_wijk_feijenoord': [0, 1],
  'adres_recentste_wijk_ijsselmonde': [0, 1],
  'adres_recentste_wijk_kralingen_c': [0, 1],
  'adres_recentste_wijk_noord': [0, 1],
  'adres_recentste_wijk_other': [0, 1],
  'adres_recentste_wijk_prins_alexa': [0, 1],
  'adres_recentste_wijk_stadscentru': [0, 1],
  'adres_unieke_wijk_ratio': [0, 1]},
 'appointment': {'afspraak_aantal_woorden': [Interval(-0.001, 230.0, closed='right'),
   Interval(230.0, 1214.0, closed='right')],
  'afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_n

In [8]:
# Create row subsets for each feature partition
def create_row_subsets_for_partitions(df, feature_partitions):
    row_subsets = {}
    for group, features in feature_partitions.items():
        row_subsets[group] = {}
        for feature, partition_values in features.items():
            row_subsets[group][feature] = {}
            for pv in partition_values:
                # If pv is a simple numeric value (e.g., 0 or 1)
                if not hasattr(pv, 'left'):   # pv is NOT an Interval (not qcut)
                    subset = df[df[feature] == pv]

                else:
                    # pv is a qcut Interval (e.g., Interval(0.1, 0.4))
                    left = pv.left
                    right = pv.right

                    subset = df[(df[feature] >= left) & (df[feature] <= right)]

                # Store subset
                row_subsets[group][feature][str(pv)] = subset

    return row_subsets

row_subsets = create_row_subsets_for_partitions(df_data, feature_partitions)
row_subsets

{'address': {'adres_recentst_onderdeel_rdam': {'0':         adres_aantal_brp_adres  adres_aantal_verschillende_wijken  \
   2                            6                                  4   
   12                           4                                  3   
   13                           3                                  3   
   14                           3                                  3   
   37                           3                                  3   
   ...                        ...                                ...   
   129872                       5                                  3   
   129949                       7                                  3   
   129950                       2                                  4   
   129964                       6                                  4   
   129996                       5                                  3   
   
           adres_aantal_verzendadres  adres_aantal_woonadres_handmatig  \
   2     

### Evaluate the row subset partitions

In [10]:
# import the model
import onnxruntime as rt
from sklearn.metrics import accuracy_score

session_gb = rt.InferenceSession("model/bad_model_histGradBoosting.onnx")
input_name_gb = session_gb.get_inputs()[0].name

# predict function
def predict_onnx(session, input_name, X):
    preds = session.run(None, {input_name: X.astype(np.float32)})[0]
    return preds.reshape(-1)  # Flatten the output array

In [11]:
# Evaluate accuracy
def evaluate_accuracy_on_subsets(row_subsets, session, input_name, label_col):
    partition_accuracies = {}

    for group, features in row_subsets.items():
        partition_accuracies[group] = {}

        for feature, partitions in features.items():
            partition_accuracies[group][feature] = {}

            for pv, subset in partitions.items():
                if subset.empty:
                    partition_accuracies[group][feature][pv] = None
                    continue
                
                # Seperate features and labels
                X = subset.drop(columns=[label_col]).values
                y = subset[label_col].values

                y_pred = predict_onnx(session, input_name, X)

                accuracy = accuracy_score(y, y_pred)

                partition_accuracies[group][feature][pv] = accuracy
            
    return partition_accuracies

In [12]:
# Calculate accuracy gaps
def compute_accuracy_gaps(partition_accuracies):
    accuracy_gaps = {}

    for group, features in partition_accuracies.items():
        accuracy_gaps[group] = {}

        for feature, partitions in features.items():
            accuracies = [acc for acc in partitions.values() if acc is not None]
            if not accuracies:
                accuracy_gaps[group][feature] = None
                continue

            max_acc = max(accuracies)
            min_acc = min(accuracies)
            gap = max_acc - min_acc

            accuracy_gaps[group][feature] = round(gap, 4)

    return accuracy_gaps

In [13]:
label_col = "checked"

partition_accuracies = evaluate_accuracy_on_subsets(row_subsets, session_gb, input_name_gb, label_col)

# Print results
for group, features in partition_accuracies.items():
    print(f"\n📌 Group: {group}")
    for feature, partitions in features.items():
        print(f"  Feature: {feature}")
        for pv, acc in partitions.items():
            if acc is not None:
                print(f"    Partition {pv}: Accuracy = {acc:.4f}")
            else:
                print(f"    Partition {pv}: No data available")


📌 Group: address
  Feature: adres_recentst_onderdeel_rdam
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 0.9998
  Feature: adres_recentste_buurt_groot_ijsselmonde
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 1.0000
  Feature: adres_recentste_buurt_nieuwe_westen
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 1.0000
  Feature: adres_recentste_buurt_other
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 0.9998
  Feature: adres_recentste_buurt_oude_noorden
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 1.0000
  Feature: adres_recentste_buurt_vreewijk
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 1.0000
  Feature: adres_recentste_plaats_other
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 1.0000
  Feature: adres_recentste_plaats_rotterdam
    Partition 0: Accuracy = 0.9998
    Partition 1: Accuracy = 0.9998
  Feature: adres_recentste_wijk_charlois
    Partition 0: Accuracy = 0.9

In [14]:
accuracy_gaps = compute_accuracy_gaps(partition_accuracies)
accuracy_gaps

{'address': {'adres_recentst_onderdeel_rdam': 0.0,
  'adres_recentste_buurt_groot_ijsselmonde': 0.0002,
  'adres_recentste_buurt_nieuwe_westen': 0.0002,
  'adres_recentste_buurt_other': 0.0001,
  'adres_recentste_buurt_oude_noorden': 0.0002,
  'adres_recentste_buurt_vreewijk': 0.0002,
  'adres_recentste_plaats_other': 0.0002,
  'adres_recentste_plaats_rotterdam': 0.0,
  'adres_recentste_wijk_charlois': 0.0,
  'adres_recentste_wijk_delfshaven': 0.0001,
  'adres_recentste_wijk_feijenoord': 0.0,
  'adres_recentste_wijk_ijsselmonde': 0.0002,
  'adres_recentste_wijk_kralingen_c': 0.0004,
  'adres_recentste_wijk_noord': 0.0002,
  'adres_recentste_wijk_other': 0.0,
  'adres_recentste_wijk_prins_alexa': 0.0004,
  'adres_recentste_wijk_stadscentru': 0.0002,
  'adres_unieke_wijk_ratio': 0.0},
 'appointment': {'afspraak_aantal_woorden': 0.0001,
  'afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel': 0.0002,
  'afspraak_afgelopen_jaar_ontheffing_taaleis'

### Save results

In [17]:
# Save json results
import json
with open("results/partition_accuracies_bad_model_histGradBoosting.json", "w") as f:
    json.dump(partition_accuracies, f, indent=4)
    
with open("results/partition_accuracy_gaps_bad_model_histGradBoosting.json", "w") as f:
    json.dump(accuracy_gaps, f, indent=4)

## Metamorphic perspective

In [29]:
import random

def metamorphic_transform_value(value, unique_values, num_unique_values, categorical_threshold=100):
    """
    Metamorphic transformation for a feature value.
    If the feature is binary (2 unique values), flip the value.
    If the feature is categorical with more than 2 unique values, randomly select a different value.
    If the feature is continuous (more than categorical_threshold unique values), perturb the value.
    Parameters:
    - value: original feature value
    - unique_values: list of unique values for the feature
    - num_unique_values: number of unique values for the feature
    - categorical_threshold: threshold to distinguish categorical from continuous features
    Returns:
    - transformed_value: metamorphically transformed feature value
    """
    
    if num_unique_values <= 2:
        # Binary feature, flip the value
        if value in [0, 1]:
            return 1 - value
        return value  # In case of unexpected binary values
    
    elif num_unique_values <= categorical_threshold:
        # Categorical feature with more than 2 unique values
        other_values = [v for v in unique_values if v != value]
        if len(other_values) == 0:
            return value  # No other value to choose from
        return random.choice(other_values)
    
    else:
        # Continuous feature, perturb the value slightly
        min_value = min(unique_values)
        max_value = max(unique_values)
        # Bounded perturbation
        delta = (max_value - min_value) * 0.05  # 5% of the range
        perturbed_value = value + random.uniform(-delta, delta)
        # Ensure perturbed value is within the original range
        perturbed_value = max(min_value, min(max_value, perturbed_value))
        return perturbed_value

In [23]:
# Function for Metamorphic transformation on each row
def transform_row_for_feature(row, feature, feature_unique_values, feature_unique_counts):
    val = row[feature]
    unique_values = feature_unique_values[feature]
    num_unique_values = feature_unique_counts[feature]

    row2 = row.copy()
    row2[feature] = metamorphic_transform_value(val, unique_values, num_unique_values)
    return row2


In [22]:
# Build dictionary of unique values and counts for each bad feature
feature_unique_values = {}
feature_unique_counts = {}
for group, features in grouped_bad_features.items():
    for feature in features:
        unique_values = df_data[feature].dropna().unique().tolist()
        feature_unique_values[feature] = unique_values
        feature_unique_counts[feature] = len(unique_values)

In [24]:
# Apply metamorphic transformation to create a new dataset and test the model
def metamorphic_test(df, bad_features, unique_values_dict, unique_count_dict, session, input_name):
    results = []

    for idx, row in df.iterrows():
        x = row.values.astype(np.float32)
        pred_x = predict_onnx(session, input_name, x.reshape(1, -1))[0]

        for feature in bad_features:
            if feature not in df.columns:
                continue

            # create x'
            row2 = transform_row_for_feature(
                row, feature, unique_values_dict, unique_count_dict
            )
            x2 = row2.values.astype(np.float32)
            pred_x2 = predict_onnx(session, input_name, x2.reshape(1, -1))[0]

            delta = abs(pred_x - pred_x2)

            results.append({
                "row": idx,
                "feature": feature,
                "pred_original": float(pred_x),
                "pred_transformed": float(pred_x2),
                "delta": float(delta)
            })

    return pd.DataFrame(results)




In [30]:
# Data without label column
df_features_only = df_data.drop(columns=[label_col])

metamorphic_results = metamorphic_test(
    df_features_only,
    bad_features,
    feature_unique_values,
    feature_unique_counts,
    session_gb,
    input_name_gb
)

In [31]:
metamorphic_results

,row,feature,pred_original,pred_transformed,delta
0,0,adres_recentst_onderdeel_rdam,0.0,0.0,0.0
1,0,adres_recentste_buurt_groot_ijsselmonde,0.0,0.0,0.0
2,0,adres_recentste_buurt_nieuwe_westen,0.0,0.0,0.0
3,0,adres_recentste_buurt_other,0.0,0.0,0.0
4,0,adres_recentste_buurt_oude_noorden,0.0,0.0,0.0
...,...,...,...,...,...
13259995,129999,relatie_overig_kostendeler,0.0,0.0,0.0
13259996,129999,relatie_partner_aantal_partner___partner__gehuwd_,0.0,0.0,0.0
13259997,129999,relatie_partner_aantal_partner___partner__onge...,0.0,0.0,0.0
13259998,129999,relatie_partner_huidige_partner___partner__geh...,0.0,0.0,0.0


In [21]:
grouped_feature_unique_values

{'address': {'adres_recentst_onderdeel_rdam': 2,
  'adres_recentste_buurt_groot_ijsselmonde': 2,
  'adres_recentste_buurt_nieuwe_westen': 2,
  'adres_recentste_buurt_other': 2,
  'adres_recentste_buurt_oude_noorden': 2,
  'adres_recentste_buurt_vreewijk': 2,
  'adres_recentste_plaats_other': 2,
  'adres_recentste_plaats_rotterdam': 2,
  'adres_recentste_wijk_charlois': 2,
  'adres_recentste_wijk_delfshaven': 2,
  'adres_recentste_wijk_feijenoord': 2,
  'adres_recentste_wijk_ijsselmonde': 2,
  'adres_recentste_wijk_kralingen_c': 2,
  'adres_recentste_wijk_noord': 2,
  'adres_recentste_wijk_other': 2,
  'adres_recentste_wijk_prins_alexa': 2,
  'adres_recentste_wijk_stadscentru': 2,
  'adres_unieke_wijk_ratio': 2},
 'appointment': {'afspraak_aantal_woorden': 967,
  'afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel': 2,
  'afspraak_afgelopen_jaar_ontheffing_taaleis': 2,
  'afspraak_galo_gesprek': 4,
  'afspraak_gespr__einde_zoekt___galo_gesprek